In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.utils import np_utils,plot_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import confusion_matrix


Using TensorFlow backend.


In [2]:
N_CLASSES = 10

In [3]:
def cnn_setup(n_classes, n_kernels = 32, kernel_size = (3,3)):
  #Instancia a rede
  cnn = Sequential()

  #Camada de convolução
  cnn.add(Conv2D(n_kernels, kernel_size,input_shape=(28, 28, 1), activation = 'relu'))

  #Normaliza os feature maps
  cnn.add(BatchNormalization())

  #camada de pooling
  cnn.add(MaxPooling2D(pool_size = (2,2)))

  #Novas camadas (opcional) de convolução e pooling
  cnn.add(Conv2D(n_kernels, kernel_size, activation = 'relu'))
  cnn.add(BatchNormalization())
  cnn.add(MaxPooling2D(pool_size = (2,2)))

  #Camada de flattening
  cnn.add(Flatten())

  #Rede densa
  n_neuronios = 128
  # percentual de neuronios que não vou atualizar os pesos
  taxa_dropout = 0.2

  #Primeira camada oculta
  cnn.add(Dense(units = n_neuronios, activation = 'relu'))
  # é para ignorar
  cnn.add(Dropout(taxa_dropout))

  #Segunda camada oculta
  cnn.add(Dense(units = n_neuronios, activation = 'relu'))
  cnn.add(Dropout(taxa_dropout))

  #Camada de saida
  cnn.add(Dense(units = n_classes, activation = 'softmax'))
  return cnn


In [4]:
def show_sample_data(inputs,classes):
  N = 6
  for i in range(N):
    plt.subplot(N,N,i+1),plt.imshow(inputs[i],cmap = 'gray')
    plt.title(classes[i]), plt.xticks([]), plt.yticks([])

  plt.show()


In [ ]:
(X_training, y_training), (X_test, y_test) = mnist.load_data()

In [ ]:
show_sample_data(X_test,y_test)

In [ ]:
db_training = X_training.reshape(X_training.shape[0], 28, 28, 1)
db_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
db_training = db_training.astype('float32')
db_test = db_test.astype('float32')
db_training /= 255
db_test /= 255

In [ ]:
classe_training = np_utils.to_categorical(y_training, N_CLASSES)
classe_test = np_utils.to_categorical(y_test, N_CLASSES)

In [ ]:
classe_training

In [ ]:
classifier = cnn_setup(N_CLASSES)

In [ ]:
classifier.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
classifier.fit(db_training, classe_training, batch_size = 128, epochs = 3, validation_data = (db_test, classe_test))

In [ ]:
print("Teste da rede:")
resultado = classifier.evaluate(db_test, classe_test)
print(resultado)

In [ ]:
print("Matriz de confusão:")
predictions = classifier.predict(db_test)
classes = [np.argmax(t) for t in classe_test]
previsoes = [np.argmax(t) for t in predictions]
conf_matrix = confusion_matrix(previsoes, classes)
print(conf_matrix)